# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [5]:
# to do: homework :)
import dlt
from dlt.sources.helpers import requests

In [7]:
pipeline = dlt.pipeline(pipeline_name="hw_pipeline1",
						destination='duckdb',
						dataset_name='hw')

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
                    table_name="question3",
                    write_disposition="replace")

# show the outcome
print(info)



Pipeline hw_pipeline1 load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw
The duckdb destination used duckdb:////content/hw_pipeline1.duckdb location to store data
Load package 1708286067.0508854 is LOADED and contains no failed jobs


In [10]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


q3 = conn.sql("SELECT * FROM question3").df()
display(q3)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ question3           │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708286067.0508854,FyNLvHKTrZ0u8Q
1,2,Person_2,27,City_A,1708286067.0508854,F6JIUJjIWenPTw
2,3,Person_3,28,City_A,1708286067.0508854,oVueHUMF1XtAIw
3,4,Person_4,29,City_A,1708286067.0508854,1sv+gYTqV2vIjQ
4,5,Person_5,30,City_A,1708286067.0508854,tIb/YBMCJSa1ww


In [11]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_2(),
                    table_name="question3",
                    write_disposition="append")

# show the outcome
print(info)

Pipeline hw_pipeline1 load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw
The duckdb destination used duckdb:////content/hw_pipeline1.duckdb location to store data
Load package 1708286765.9492986 is LOADED and contains no failed jobs


In [22]:
import duckdb

conn = duckdb.connect("hw_pipeline1.duckdb")
conn.sql("SET search_path = 'hw'")
print('Loaded tables: ')
display(conn.sql("show tables"))


q3 = conn.sql("SELECT * FROM question3").df()
q3_1 = conn.sql("SELECT SUM(age) AS total_age FROM question3").df()
display(q3)
display(q3_1)



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ question3           │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708286067.0508854,FyNLvHKTrZ0u8Q,None
1,2,Person_2,27,City_A,1708286067.0508854,F6JIUJjIWenPTw,None
2,3,Person_3,28,City_A,1708286067.0508854,oVueHUMF1XtAIw,None
3,4,Person_4,29,City_A,1708286067.0508854,1sv+gYTqV2vIjQ,None
4,5,Person_5,30,City_A,1708286067.0508854,tIb/YBMCJSa1ww,None
5,3,Person_3,33,City_B,1708286765.9492986,K+8IE74VNgfq4g,Job_3
6,4,Person_4,34,City_B,1708286765.9492986,VW6Tni+WYS3Q6w,Job_4
7,5,Person_5,35,City_B,1708286765.9492986,bPOQq73SFpPZ6A,Job_5
8,6,Person_6,36,City_B,1708286765.9492986,qP0c6eSgoLz8Qw,Job_6
9,7,Person_7,37,City_B,1708286765.9492986,6n2HLn4+apWyxg,Job_7


,total_age
0,353.0


In [14]:
pipeline = dlt.pipeline(pipeline_name="hw_pipeline2",
						destination='duckdb',
						dataset_name='hw')

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}



# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
                    table_name="question4",
                    write_disposition="replace",primary_key="id")

# show the outcome
print(info)


Pipeline hw_pipeline2 load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw
The duckdb destination used duckdb:////content/hw_pipeline2.duckdb location to store data
Load package 1708287050.7158682 is LOADED and contains no failed jobs


In [15]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_2(),
                    table_name="question4",
                    write_disposition="merge")

# show the outcome
print(info)

Pipeline hw_pipeline2 load step completed in 0.59 seconds
1 load package(s) were loaded to destination duckdb and into dataset hw
The duckdb destination used duckdb:////content/hw_pipeline2.duckdb location to store data
Load package 1708287082.70673 is LOADED and contains no failed jobs


In [18]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


q4 = conn.sql("SELECT * FROM question4").df()
q4_1 = conn.sql("SELECT SUM(age) AS total_age FROM question4").df()
display(q4)
display(q4_1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ question4           │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708287050.7158682,rpMK5DVec4FEAw,None
1,2,Person_2,27,City_A,1708287050.7158682,BAKIXVC//WUpHQ,None
2,8,Person_8,38,City_B,1708287082.70673,oAxFlBdWIYI6aA,Job_8
3,4,Person_4,34,City_B,1708287082.70673,pA235oXy2YZZOA,Job_4
4,5,Person_5,35,City_B,1708287082.70673,1aRDiyLSRMBtwg,Job_5
5,7,Person_7,37,City_B,1708287082.70673,5R2+d9yIbgoxQA,Job_7
6,3,Person_3,33,City_B,1708287082.70673,ErpRq+nm7yyNJQ,Job_3
7,6,Person_6,36,City_B,1708287082.70673,Cfn2MAeH7f4Fyg,Job_6


,total_age
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX